In [35]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [37]:
url_inicial='http://alhe.mora.edu.mx/index.php/ALHE'
url_root= 'http://alhe.mora.edu.mx/index.php/ALHE'
r = requests.get(url_inicial)

In [38]:
soup = BeautifulSoup(r.text, 'html.parser')

In [39]:
lista_articulo = soup.find_all('div', class_='media-body')
links_libros = [x.find('a').get('href') for x in lista_articulo]
links_libros = [urljoin (url_root, i) for i in links_libros]
links_libros

['http://alhe.mora.edu.mx/index.php/ALHE/article/view/1279',
 'http://alhe.mora.edu.mx/index.php/ALHE/article/view/1230',
 'http://alhe.mora.edu.mx/index.php/ALHE/article/view/1259',
 'http://alhe.mora.edu.mx/index.php/ALHE/article/view/1280',
 'http://alhe.mora.edu.mx/index.php/ALHE/article/view/1287',
 'http://alhe.mora.edu.mx/index.php/ALHE/article/view/1289',
 'http://alhe.mora.edu.mx/index.php/ALHE/article/view/1290',
 'http://alhe.mora.edu.mx/index.php/ALHE/article/view/1208']

In [40]:
def get_url_items(soup,url):
    lista_articulo = soup.find_all('div', class_='media-body')
    links_libros = [x.find('a').get('href') for x in lista_articulo]
    links_libros = [urljoin (url_root, i) for i in links_libros]
    return links_libros

In [41]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina http://alhe.mora.edu.mx/index.php/ALHE


In [42]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

8

In [43]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [54]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='América Latina en la Historia Económica'
    a='IV. Humanidades y Ciencias de la Conducta'
    tem='Historia Económica'
    #url='http://alhe.mora.edu.mx/'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo libro
    try:
        titu=s_item.find('h1', class_='page-header').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='article-abstract').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('a', rel='license').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        link=s_item.find('div', class_='list-group-item doi').find('a').get('href')
        content_book['Link Articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('div', class_='download').find(role="button").get('data-href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [57]:
list_scraper=list_scraper[0:8]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7


In [58]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,América Latina en la Historia Económica,IV. Humanidades y Ciencias de la Conducta,Historia Económica,El diario Noticias: ideas económicas de la izq...,En el presente trabajo proponemos un examen in...,,https://doi.org/10.18232/20073496.1279,http://alhe.mora.edu.mx/index.php/ALHE
1,América Latina en la Historia Económica,IV. Humanidades y Ciencias de la Conducta,Historia Económica,Comparación del índice de salarios reales de u...,"Real del Monte, durante los siglos XVIII y XIX...",,https://doi.org/10.18232/20073496.1230,http://alhe.mora.edu.mx/index.php/ALHE
2,América Latina en la Historia Económica,IV. Humanidades y Ciencias de la Conducta,Historia Económica,"El mercado musical italiano en Argentina, 1895...",El artículo tiene por objetivo estudiar un mer...,,https://doi.org/10.18232/20073496.1259,http://alhe.mora.edu.mx/index.php/ALHE
3,América Latina en la Historia Económica,IV. Humanidades y Ciencias de la Conducta,Historia Económica,La fruticultura en el Alto Valle de Río Negro:...,La producción frutícola ubicada en la región d...,,https://doi.org/10.18232/20073496.1280,http://alhe.mora.edu.mx/index.php/ALHE
4,América Latina en la Historia Económica,IV. Humanidades y Ciencias de la Conducta,Historia Económica,La desindustrialización argentina en el largo ...,En 1974 la economía argentina acumulaba una dé...,,https://doi.org/10.18232/20073496.1287,http://alhe.mora.edu.mx/index.php/ALHE
5,América Latina en la Historia Económica,IV. Humanidades y Ciencias de la Conducta,Historia Económica,"Estado, política externa e internacionalização...",O presente artigo busca compreender o início d...,,https://doi.org/10.18232/20073496.1289,http://alhe.mora.edu.mx/index.php/ALHE
6,América Latina en la Historia Económica,IV. Humanidades y Ciencias de la Conducta,Historia Económica,Continuidades y rupturas de la cúpula empresar...,En el presente trabajo se buscará poner en evi...,,https://doi.org/10.18232/20073496.1290,http://alhe.mora.edu.mx/index.php/ALHE
7,América Latina en la Historia Económica,IV. Humanidades y Ciencias de la Conducta,Historia Económica,Foreign Direct Investment and Multinationals i...,"Foreign direct investment (FDI) in Brazil, sta...",,https://doi.org/10.18232/20073496.1208,http://alhe.mora.edu.mx/index.php/ALHE


In [59]:
df_catalogo.to_csv('Revista07.csv', index=False)